# Bomberman Tower ML-Agents Training

Bu notebook Bomberman Tower oyunu için ML-Agents training'ini başlatır ve izler.

## 1. Setup ve Imports

In [1]:
import os
import sys
import subprocess
import time
import threading
from datetime import datetime

# ML-Agents imports
try:
    from mlagents.trainers.learn import run_cli
    print("✅ ML-Agents imported successfully")
except ImportError:
    print("❌ ML-Agents not found. Install with: pip install mlagents")
    
# Set environment variable for protobuf compatibility
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
print(f"📁 Current directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## 2. Training Configuration

In [ ]:
# Training parameters
CONFIG_FILE = "config/bomberman_ppo_simple.yaml"
RUN_ID = f"bomberman_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
MAX_STEPS = 10000  # Training için adım sayısı
FORCE_OVERWRITE = True

print(f"🎯 Config file: {CONFIG_FILE}")
print(f"🆔 Run ID: {RUN_ID}")
print(f"🔢 Max steps: {MAX_STEPS}")

# Check if config file exists
if os.path.exists(CONFIG_FILE):
    print(f"✅ Config file found: {CONFIG_FILE}")
else:
    print(f"❌ Config file not found: {CONFIG_FILE}")

## 3. Start Training Server

Bu cell'i çalıştır ve Unity'de Play'e bas!

In [ ]:
def start_training():
    """Start ML-Agents training server"""
    
    # Build command arguments
    cmd_args = [
        "python", "-m", "mlagents.trainers.learn",
        CONFIG_FILE,
        f"--run-id={RUN_ID}"
    ]
    
    if FORCE_OVERWRITE:
        cmd_args.append("--force")
    
    print(f"🚀 Starting training with command:")
    print(f"   {' '.join(cmd_args)}")
    print(f"")
    print(f"📋 Training info:")
    print(f"   - Config: {CONFIG_FILE}")
    print(f"   - Run ID: {RUN_ID}")
    print(f"   - Max steps: {MAX_STEPS}")
    print(f"   - Results will be saved to: results/{RUN_ID}")
    print(f"")
    print(f"🎮 Now go to Unity and press PLAY!")
    print(f"   You should see: 'Connected to Unity environment'")
    print(f"")
    print(f"📊 To monitor training:")
    print(f"   - Watch the output below")
    print(f"   - Or run: tensorboard --logdir results")
    print(f"")
    print(f"⛔ To stop training: Interrupt the kernel (Kernel > Interrupt)")
    print(f"" + "="*60)
    
    try:
        # Start the training process
        process = subprocess.Popen(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
            bufsize=1
        )
        
        # Stream output in real-time
        for line in process.stdout:
            print(line.rstrip())
            
        process.wait()
        
        if process.returncode == 0:
            print(f"\n✅ Training completed successfully!")
        else:
            print(f"\n❌ Training failed with return code: {process.returncode}")
            
    except KeyboardInterrupt:
        print(f"\n⏹️ Training stopped by user")
        if 'process' in locals():
            process.terminate()
    except Exception as e:
        print(f"\n❌ Error during training: {e}")

# Start training
start_training()

## 4. Training Monitoring

In [ ]:
# Check training results
results_dir = f"results/{RUN_ID}"

if os.path.exists(results_dir):
    print(f"📊 Training results in: {results_dir}")
    
    # List files in results directory
    for root, dirs, files in os.walk(results_dir):
        level = root.replace(results_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")
else:
    print(f"❌ No results found in: {results_dir}")

## 5. TensorBoard Visualization

In [ ]:
def start_tensorboard():
    """Start TensorBoard for training visualization"""
    
    print("🔥 Starting TensorBoard...")
    print("📊 Open browser and go to: http://localhost:6006")
    print("⏹️ To stop: Interrupt this cell")
    print("="*50)
    
    try:
        process = subprocess.Popen([
            "tensorboard", "--logdir", "results",
            "--host", "localhost",
            "--port", "6006"
        ])
        
        print(f"🟢 TensorBoard started with PID: {process.pid}")
        process.wait()
        
    except KeyboardInterrupt:
        print("\n⏹️ TensorBoard stopped")
        if 'process' in locals():
            process.terminate()
    except FileNotFoundError:
        print("❌ TensorBoard not found. Install with: pip install tensorboard")
    except Exception as e:
        print(f"❌ Error starting TensorBoard: {e}")

# Uncomment to start TensorBoard
# start_tensorboard()

## 6. Quick Training Commands

In [ ]:
# Quick commands for different training scenarios

def quick_train(steps=1000, run_suffix=""):
    """Quick training with specified steps"""
    quick_run_id = f"quick_train_{steps}{run_suffix}_{datetime.now().strftime('%H%M%S')}"
    
    cmd = f"""python -m mlagents.trainers.learn {CONFIG_FILE} --run-id={quick_run_id} --force"""
    
    print(f"🏃‍♂️ Quick training ({steps} steps):")
    print(f"   Run ID: {quick_run_id}")
    print(f"   Command: {cmd}")
    print(f"\n🎮 Go to Unity and press PLAY!")
    
    return cmd

# Generate quick commands
print("⚡ Quick Training Options:")
print("1. Test (1K steps):", quick_train(1000, "_test"))
print("2. Short (5K steps):", quick_train(5000, "_short"))
print("3. Medium (10K steps):", quick_train(10000, "_medium"))
print("\n💡 Copy-paste any command above to terminal if needed")